Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [6]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [7]:
import pandas as pd
# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [8]:
import category_encoders as ce
import numpy as np
from scipy.stats import randint, uniform
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [31]:
def wrangler(dataframe):

  x = dataframe.copy()
  
  # New Feature
  x['date_recorded_year'] = x['date_recorded'].str[:4].astype(int)
  x['years_before_service'] = x['date_recorded_year'] - x['construction_year']
  x.loc[(x['years_before_service']<0) | (x['years_before_service']>100), 'years_before_service'] = np.nan

  # Replace None, none, 0 with NaN values, and fix long/lat columns
  features_replace = ['scheme_name', 'installer', 'funder', 'wpt_name', 'longitude', 'latitude']
  x[features_replace] = x[features_replace].replace({'None':np.nan, 'none':np.nan, '0':np.nan, 0:np.nan, -2e-8:np.nan})
  
  # Drop id
  drops = ['id', 'recorded_by']
  x = x.drop(drops, axis=1)

  return x

In [25]:
wrangled_train = wrangler(train)

In [27]:
target = 'status_group'
features = wrangled_train.columns.drop(target)
X_train = wrangled_train[features]
y_train = wrangled_train[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    StandardScaler(),
    RandomForestClassifier(random_state=42)
)

param_distributions = {
    'ordinalencoder__handle_unknown': ['return_nan', 'value'], 
    'ordinalencoder__handle_missing': ['return_nan', 'value'], 
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'], 
    'randomforestclassifier__n_estimators': randint(20, 1000), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, 30, None], 
    'randomforestclassifier__max_features': uniform(0, 1), 
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train)
print(search.best_params_)
print(search.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  30 | elapsed:   15.4s remaining:  2.3min
[Parallel(n_jobs=-1)]: Done   7 out of  30 | elapsed:  1.2min remaining:  3.9min
[Parallel(n_jobs=-1)]: Done  11 out of  30 | elapsed:  1.7min remaining:  2.9min
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:  3.1min remaining:  3.1min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:  3.4min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:  3.9min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  6.3min remaining:   42.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  6.9min finished


{'ordinalencoder__handle_missing': 'value', 'ordinalencoder__handle_unknown': 'return_nan', 'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 0.0803875104989975, 'randomforestclassifier__n_estimators': 962, 'simpleimputer__strategy': 'mean'}
0.8095622895622896


In [ ]:
wrangled_train.columns

In [15]:
{'ordinalencoder__handle_missing': 'value', 'ordinalencoder__handle_unknown': 'return_nan', 'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 0.1461191250901842, 'randomforestclassifier__n_estimators': 915, 'simpleimputer__strategy': 'mean'}
0.8092087542087542

0.8082323232323232

In [28]:
new_pipe = search.best_estimator_
X_test = wrangler(test)
y_pred = new_pipe.predict(X_test)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission1.csv', index=False)

## Let's get weird

In [29]:
param_distributions1 = {
    'ordinalencoder__handle_unknown': ['return_nan', 'value'], 
    'ordinalencoder__handle_missing': ['return_nan', 'value'], 
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'], 
    'randomforestclassifier__n_estimators': randint(20, 1500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, 30, None], 
    'randomforestclassifier__max_features': uniform(0, 1), 
}

search1 = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions1, 
    n_iter=20, 
    cv=5, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search1.fit(X_train, y_train)
print(search1.best_params_)
print(search1.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 15.1min
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done  80 out of 100 | elapsed: 20.2min remaining:  5.1min
[Parallel(n_jobs=-1)]: Done  91 out of 100 | elapsed: 25.1min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 30.9min finished


{'ordinalencoder__handle_missing': 'value', 'ordinalencoder__handle_unknown': 'return_nan', 'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 0.448029758581302, 'randomforestclassifier__n_estimators': 215, 'simpleimputer__strategy': 'median'}
0.810959595959596


In [33]:
new_pipe = search1.best_estimator_
X_test = wrangler(test)
y_pred = new_pipe.predict(X_test)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission2.csv', index=False)

It made it worse on the kaggle leaderboard ;( there goes 30.9minutes